In [22]:
import pandas as pd
import numpy as np
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import interactive

import folium
import folium.plugins as plugins
from folium.plugins import HeatMap

# Read in data
df = pd.read_csv("../../project_datasets/Collisions_weather_iem_day_2014_2018_fei.csv")

In [90]:
df.weather.value_counts()

Clear or Partly Cloudy          27662
Raining                          8396
Overcast                         6281
Unknown                          4154
Other                             149
Snowing                           143
Fog/Smog/Smoke                    138
Sleet/Hail/Freezing Rain           14
Blowing Sand or Dirt or Snow        8
Severe Crosswind                    7
Name: weather, dtype: int64

In [3]:
# As shown above, rearranging of the weather column may be necessary. For instance merge NULL with unknown, 
# shorten the levels with too many words.

#### Example Code to create an interactice widget

In [38]:
# Define widget
weather_selection = widgets.Dropdown(
    options=list(set(df[-pd.isnull(df.weather)].weather.values)),
    value='Snowing',
    description='Weather:',
    disabled=False
)

# Define function to view or plot data
def view(weather=''):
     print(df[df.weather == weather].shape)

# Define interactivity
interactive(view, weather=weather_selection)

interactive(children=(Dropdown(description='Weather:', index=5, options=('Blowing Sand or Dirt or Snow', 'Seve…

#### Create an interactive widget to plot heatmap of collisions by weather type

In [89]:
# Define widget
weather_selection = widgets.Dropdown(
    options=list(set(df[-pd.isnull(df.weather)].weather.values)),
    value='Snowing',
    description='Weather:',
    disabled=False
)

columns = ['X', 'Y', 'weather']

# Define function to view or plot data
def map_by_weather(weather=''):
    df_collision = df.loc[df.weather == weather, columns].dropna(axis = 0, how = 'any')
    coordinates = df_collision.loc[:, ['Y', 'X']].values
    NewData = coordinates * np.array([[1, 1]])
    
    data = list()
    data.append(NewData.tolist())
    m = folium.Map(location=[47.706850, -122.333961], 
                   tiles='Stamen Toner', 
                   zoom_start=11)
    HeatMap(data[0]).add_to(m)
    display(m)
    
# Define interactivity
interactive(map_by_weather, weather=weather_selection)

interactive(children=(Dropdown(description='Weather:', index=5, options=('Blowing Sand or Dirt or Snow', 'Seve…

#### TO-DO: toggle by neighborhoods

In [ ]:
def visualize_neighborhood(neighborhood_data, mapping_value):
    """
    Visualizes the data provided per neighborhood in abs
    folim map nad returns the map.
    Args:
        neighborhood_data(pandas dataframe): Dataframe containing
            the data value per neighborhood which is to be mapped.
        mapping_value (string): Name of column of neighborhood_data
            to be mapped.
    Returns:
        the map produced
    Raises:
        ValueError: if neighborhood_data doesn't have the column
            'object_id' or mapping_value
    """
    if not 'object_id' in neighborhood_data.columns:
        raise ValueError("Dataframe doesn't have a column object_id")
    if not mapping_value in neighborhood_data.columns:
        raise ValueError("Dataframe doesn't have a column " + mapping_value)

    neigborhood_map = folium.Map(
        location=MAP_LOCATION_START,
        zoom_start=MAP_ZOOM)

    # Finding maximum and minimum values for range
    # Adding a small error to max as folium is not good at comparing floats
    max_value = max(neighborhood_data[mapping_value]) + ERROR
    min_value = min(neighborhood_data[mapping_value])

    neigborhood_map.choropleth(
        geo_data=MAP_JSON,
        data=neighborhood_data,
        columns=['object_id', mapping_value],
        key_on='feature.properties.OBJECTID',
        threshold_scale=list(np.linspace(min_value, max_value, 6)),
        fill_color='YlOrRd',
        highlight=True)
    return neigborhood_map